In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as m
from torchsummary import summary

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 1.7.1
GPU found :)


In [10]:
EMBEDDING_SIZE = 300

In [23]:
# retrieve pretrained model for features extraction
alexnet = m.alexnet().to(device)
summary(alexnet, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [24]:
# Keep only the feature extraction layers of the model
features = nn.Sequential(*(list(alexnet.children())[0]))
summary(features, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
Total params: 2,469,696
Trainable param

## Build LSTM 

In [32]:
class LSTMCaptioning(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        self.lstm = nn.LSTM(input_size, hidden_size)
        
    def forward(self, inputs):
         
        # Get hidden states for each t (out) , and latest one (h = (ht, ct))
        out, ht = self.lstm(inputs.view(len(inputs), 1, -1))
        
        # Compute probability distribution over all words for this t
        pt = F.softmax(ht, dim=1)
                           
        return out, ht, pt
        

## Train model

In [ ]:
input_size = (3, 224, 224)
hideen_size = ...

model = LSTMCaptioning(input_size, hidden_size)

num_epoch = 10
loss_function = nn.NLLLoss()

for epoch in range(num_epoch):
    for image, caption in dataset:
        
        # Reset grad
        model.zero_grad()
        
        # Get the input image embedding 
        embedding = features(image)
        targets = ...
        
        # Forward pass
        out, hidden, probs = model(embedding)
        
        # Compute loss and backprop
        loss = loss_function(out, targets)
        loss.backward()
        optimizer.step()
        

In [ ]:
# Save model 
torch.save(model.state_dict(), './models/')